In [10]:
import os
os.environ['SPARK_HOME'] = "C:/Users/jonat/Jupyter/spark-3.2.1-bin-hadoop2.7"

In [4]:
!pip install -q findspark

In [13]:
import findspark
findspark.init()

In [14]:
#criar spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('sparkjupyter')\
        .getOrCreate()

In [19]:
#importar o dataset
df = spark.read.csv('C:/Users/jonat/Jupyter/House_Sales.csv', header=True, inferSchema=True)

In [20]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)



In [26]:
#selecinar colunas
df_casas = df.select('date','price','bedrooms','bathrooms','condition','grade')
df_casas.show()

+---------------+---------+--------+---------+---------+-----+
|           date|    price|bedrooms|bathrooms|condition|grade|
+---------------+---------+--------+---------+---------+-----+
|20141013T000000| 221900.0|       3|      1.0|        3|    7|
|20141209T000000| 538000.0|       3|     2.25|        3|    7|
|20150225T000000| 180000.0|       2|      1.0|        3|    6|
|20141209T000000| 604000.0|       4|      3.0|        5|    7|
|20150218T000000| 510000.0|       3|      2.0|        3|    8|
|20140512T000000|1225000.0|       4|      4.5|        3|   11|
|20140627T000000| 257500.0|       3|     2.25|        3|    7|
|20150115T000000| 291850.0|       3|      1.5|        3|    7|
|20150415T000000| 229500.0|       3|      1.0|        3|    7|
|20150312T000000| 323000.0|       3|      2.5|        3|    7|
|20150403T000000| 662500.0|       3|      2.5|        3|    8|
|20140527T000000| 468000.0|       2|      1.0|        4|    7|
|20140528T000000| 310000.0|       3|      1.0|        4

In [28]:
#mudar tipo da coluna
df_novo = df_casas.withColumn('banheiros', df_casas['bathrooms'].cast('int')).drop('bathrooms')
df_novo.show()

+---------------+---------+--------+---------+-----+---------+
|           date|    price|bedrooms|condition|grade|banheiros|
+---------------+---------+--------+---------+-----+---------+
|20141013T000000| 221900.0|       3|        3|    7|        1|
|20141209T000000| 538000.0|       3|        3|    7|        2|
|20150225T000000| 180000.0|       2|        3|    6|        1|
|20141209T000000| 604000.0|       4|        5|    7|        3|
|20150218T000000| 510000.0|       3|        3|    8|        2|
|20140512T000000|1225000.0|       4|        3|   11|        4|
|20140627T000000| 257500.0|       3|        3|    7|        2|
|20150115T000000| 291850.0|       3|        3|    7|        1|
|20150415T000000| 229500.0|       3|        3|    7|        1|
|20150312T000000| 323000.0|       3|        3|    7|        2|
|20150403T000000| 662500.0|       3|        3|    8|        2|
|20140527T000000| 468000.0|       2|        4|    7|        1|
|20140528T000000| 310000.0|       3|        4|    7|   

In [29]:
#filtrando
df_novo.filter(df_novo.price > 700000).show()

+---------------+---------+--------+---------+-----+---------+
|           date|    price|bedrooms|condition|grade|banheiros|
+---------------+---------+--------+---------+-----+---------+
|20140512T000000|1225000.0|       4|        3|   11|        4|
|20140826T000000|2000000.0|       3|        3|    9|        2|
|20140626T000000| 937000.0|       3|        3|    8|        1|
|20150302T000000| 719000.0|       4|        3|    8|        2|
|20140811T000000| 775000.0|       4|        3|    8|        2|
|20140707T000000| 861990.0|       5|        3|    9|        2|
|20140716T000000| 785000.0|       4|        4|    9|        2|
|20150311T000000|1350000.0|       3|        5|    9|        2|
|20150303T000000| 920000.0|       5|        3|    8|        2|
|20140512T000000| 885000.0|       4|        3|    9|        2|
|20140827T000000| 951000.0|       5|        4|    8|        3|
|20140909T000000| 975000.0|       4|        3|   10|        2|
|20140612T000000|1325000.0|       5|        3|    8|   

In [30]:
#A casa mais cara
from pyspark.sql.functions import max
df_maior = df_novo.agg(max('price').alias('maior_preco')).show()

+-----------+
|maior_preco|
+-----------+
|  7700000.0|
+-----------+



In [32]:
#caracteristicas da casa mais cara
df_maior_preco = df_novo.filter(df_novo.price == 7700000.0 )
df_maior_preco.show()

+---------------+---------+--------+---------+-----+---------+
|           date|    price|bedrooms|condition|grade|banheiros|
+---------------+---------+--------+---------+-----+---------+
|20141013T000000|7700000.0|       6|        4|   13|        8|
+---------------+---------+--------+---------+-----+---------+



In [41]:
from pyspark.sql.functions import count, col, asc, desc
df_soma = df_novo.groupBy('price').count()
df_soma.show(truncate=False)

+---------+-----+
|price    |count|
+---------+-----+
|330000.0 |119  |
|300000.0 |133  |
|495000.0 |70   |
|532000.0 |9    |
|1185000.0|8    |
|452500.0 |4    |
|431500.0 |2    |
|3418800.0|1    |
|751750.0 |1    |
|217000.0 |11   |
|502550.0 |1    |
|814950.0 |1    |
|156601.0 |1    |
|352450.0 |1    |
|475200.0 |1    |
|804100.0 |1    |
|326250.0 |1    |
|640500.0 |2    |
|147000.0 |2    |
|649500.0 |2    |
+---------+-----+
only showing top 20 rows



In [42]:
df_novo.orderBy(col('price').desc()).show()

+---------------+---------+--------+---------+-----+---------+
|           date|    price|bedrooms|condition|grade|banheiros|
+---------------+---------+--------+---------+-----+---------+
|20141013T000000|7700000.0|       6|        4|   13|        8|
|20140611T000000|7062500.0|       5|        3|   11|        4|
|20140919T000000|6885000.0|       6|        3|   13|        7|
|20140804T000000|5570000.0|       5|        3|   13|        5|
|20150413T000000|5350000.0|       5|        3|   12|        5|
|20150413T000000|5300000.0|       6|        4|   12|        6|
|20141020T000000|5110800.0|       5|        3|   12|        5|
|20140617T000000|4668000.0|       5|        3|   12|        6|
|20140815T000000|4500000.0|       5|        3|   12|        5|
|20140618T000000|4489000.0|       4|        3|   12|        3|
|20150506T000000|4208000.0|       5|        3|   12|        6|
|20141014T000000|4000000.0|       4|        3|   12|        5|
|20141114T000000|3850000.0|       4|        4|   11|   

In [80]:
#convertendo a colunada date de doble para data
from pyspark.sql.functions import *
from pyspark.sql.types import DateType

df_data = df_novo.withColumn("date",df_novo['date'].cast(DateType()))
df_data.printSchema()


root
 |-- date: date (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- banheiros: integer (nullable = true)



In [48]:
#utilizando SQL
df_novo.createOrReplaceTempView('casas')
spark.sql('select date, count(*) as qtd from casas GROUP BY date ORDER BY qtd desc').show(truncate=False)

+---------------+---+
|date           |qtd|
+---------------+---+
|20140623T000000|142|
|20140626T000000|131|
|20140625T000000|131|
|20140708T000000|127|
|20150427T000000|126|
|20150325T000000|123|
|20150414T000000|121|
|20140709T000000|121|
|20150422T000000|121|
|20150428T000000|121|
|20140624T000000|120|
|20150421T000000|119|
|20141028T000000|117|
|20140826T000000|117|
|20140714T000000|116|
|20140520T000000|116|
|20140701T000000|115|
|20140820T000000|114|
|20150429T000000|114|
|20140617T000000|113|
+---------------+---+
only showing top 20 rows

